# Convert Raster Data to DGGS Zones

In [11]:
import os

import folium
import json
import pystac_client
import dggrid4py
import geopandas as gpd
import numpy as np
import shapely
import rasterio as rio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from tqdm.autonotebook import tqdm

from vgrid.conversion.dggs2geo.dggrid2geo import dggrid2geo
from vgrid.conversion.raster2dggs import raster2a5, raster2dggrid, raster2rhealpix, raster2h3
from vgrid.conversion.raster2dggs.raster2dggrid import get_nearest_dggrid_resolution
from vgrid.conversion.latlon2dggs import latlon2dggrid
from vgrid.utils.geometry import geodesic_dggs_metrics
from vgrid.utils.io import (
    convert_to_output_format,
    create_dggrid_instance,
    download_file,
    validate_dggrid_type,
    validate_dggrid_resolution,
)

In [12]:
# [optional] extra dependencies for nicer notebook progress bars if not available
!pip install -U tqdm ipywidgets jupyterlab_widgets

## Identify the Study Area of Interest

In [13]:
with open("../manitoba-winnipeg_study_area/inputs/manitoba-winnipeg_study_area.geojson") as f:
    data = f.read()
    poly = json.loads(data)
    geom = shapely.from_geojson(data)

map = folium.Map(location=[geom.centroid.y, geom.centroid.x], zoom_start=8)
folium.GeoJson(poly).add_to(map)
map

## Find RCM ARD Data from EODMS matching AOI

In [14]:
client = pystac_client.Client.open("https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/")
result = client.search(
    collections=["rcm-ard"],
    intersects=geom,
    datetime=["2018-01-01", "2026-01-01"],
    limit=100,
)
result.matched()

89

In [15]:
item = next(result.items())
item

<Item id=d1b8e42b-701e-4ae1-841a-65e11e06ce0c>

## Define DGGRID Instance (optional)

This is needed only if you want to use `raster2dggrid` conversion function.

⚠️ Below code requires https://github.com/allixender/dggrid4py/pull/26 (i.e.: `DGGRIDv8`) to resolve DGGRID system indexing issues.

In [16]:
DGGRID_EXE = os.getenv("DGGRID_EXE", "../../DGGRID/build/src/apps/dggrid/dggrid")
# dggrid_instance = create_dggrid_instance(DGGRID_EXE) # FIX: create ourselves to avoid legacy errors and add GDAL support
dggrid_instance = dggrid4py.DGGRIDv8(
    executable=DGGRID_EXE,
    working_dir='.',
    capture_logs=True,
    has_gdal=True,
    tmp_geo_out_legacy=False,
    silent=False,
    debug=False,
)

#### Utility Functions

In [17]:
def raster_reproject(raster_path: os.PathLike, dst_crs: str = "EPSG:4326") -> os.PathLike:
    reproject_name, reproject_ext = os.path.splitext(os.path.basename(raster_path))
    reproject_name = f"{reproject_name}_{dst_crs.replace(':', '')}"
    reproject_path = os.path.join(os.path.dirname(raster_path), f"{reproject_name}{reproject_ext}")
    if os.path.isfile(reproject_path):
        return reproject_path

    with rio.open(raster_path) as src:
        if src.crs == dst_crs:
            return raster_path

        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
        dst_meta = src.meta.copy()
        dst_meta.update({
            "crs": dst_crs,
            "transform": transform,
            "width": width,
            "height": height,
            # Cloud-Optimized GeoTIFF (COG) creation options
            "interleave": "pixel",
            "compress": "DEFLATE",
            "tiled": True,
            "blockxsize": 512,
            "blockysize": 512,
        })
        with rio.open(reproject_path, 'w', **dst_meta) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                )
    return reproject_path


def build_overviews(raster_path: os.PathLike, overviews: list[int] = None):
    overviews = overviews or [2, 4, 8, 16, 32]
    with rio.open(raster_path, "r+") as src:
        src.build_overviews(overviews, Resampling.nearest)
        src.update_tags(ns="rio_overview", resampling="nearest")


def raster2dggs_custom(
    dggrid_instance: "dggrid4py.DGGRID",
    dggs_type: str,
    raster_path: os.PathLike,
    resolution: int | None = None,
    output_format: str = "gpd",
    output_cell_path: str | None = None,  # raster path by default if not specified
    output_cell_data: bool = True,
    **output_kwargs,
):
    dggs_type = validate_dggrid_type(dggs_type)
    raster_path = raster_reproject(raster_path)  # pre-reproject to reduce time for each iter of dggrid
    output_cell_path = output_cell_path or os.path.dirname(raster_path)
    os.makedirs(output_cell_path, exist_ok=True)

    # Auto-select resolution if not provided
    if resolution is None:
        cell_size, resolution = get_nearest_dggrid_resolution(dggrid_instance, dggs_type, raster_path)
        print(f"Cell size: {cell_size} m2")
        print(f"Nearest {dggs_type.upper()} resolution determined: {resolution}")
    else:
        resolution = validate_dggrid_resolution(dggs_type, resolution)

    # Open the raster file to get metadata and data
    ### build_overviews(raster_path)  # FIXME: any advantage to use overviews over computing statistics over raw data directly?
    src = rio.open(raster_path)

    # collect cells
    raster_geom = shapely.geometry.box(*list(src.bounds))
    aoi_geom = shapely.intersection(raster_geom, geom)
    cells = dggrid_instance.grid_cell_polygons_for_extent(
        dggs_type,
        resolution=resolution,
        clip_geom=aoi_geom,
        **output_kwargs
    )
    cell_count = len(cells)
    print(f"Matched cells: {cell_count} (res={resolution}) in {aoi_geom}")

    properties = []

    # Collect band values during the pixel scan, storing the first sample per DGGRID cell
    for cell in tqdm(
        cells.itertuples(index=False),
        desc="Converting raster to DGGRID",
        unit=" cells",
        total=cell_count,
    ):
        cell_id = cell.name if hasattr(cell, "name") else cell.global_id
        cell_geom = cell.geometry
        try:
            cell_data, cell_transform = mask(src, [cell_geom], crop=True)
        except ValueError:
            continue  # skip empty cells (geom does not overlap raster)
        cell_meta = src.meta.copy()
        cell_meta.update({
            "driver": "GTiff",
            "transform": cell_transform,
            "crs": src.crs,
            "height": cell_data.shape[1],
            "width": cell_data.shape[2],
            # Cloud-Optimized GeoTIFF (COG) creation options
            "interleave": "pixel",
            "compress": "DEFLATE",
            "tiled": True,
            "blockxsize": 512,
            "blockysize": 512,
        })

        cell_valid_data = cell_data[cell_data != src.nodata]
        if len(cell_valid_data) == 0:
            continue  # skip empty cells

        if output_cell_data:
            output_cell_file = os.path.join(output_cell_path, f"cell-{cell_id}.tif")
            with rio.open(output_cell_file, "w", **cell_meta) as cell_f:
                cell_f.write(cell_data)

        cell_mean = np.mean(cell_valid_data)
        cell_stddev = np.std(cell_valid_data)
        cell_median = np.median(cell_valid_data)
        cell_minimum = np.min(cell_valid_data)
        cell_maximum = np.max(cell_valid_data)
        cell_pixel_count = len(cell_valid_data)

        # Build GeoDataFrame as the base
        try:
            # Get cell metrics
            #if isinstance(cell_polygon, gpd.GeoDataFrame) and not cell_geom.empty:
            centroid_lat, centroid_lon, avg_edge_len, cell_area, cell_perimeter = geodesic_dggs_metrics(
                cell_geom, len(cell_geom.exterior.coords) - 1
            )
            # else:
            #     # Fallback metrics if geometry conversion fails
            #     centroid_lat, centroid_lon, avg_edge_len, cell_area, cell_perimeter = 0, 0, 0, 0, 0

            cell_props = {
                f"dggrid_{dggs_type}": cell_id,
                "resolution": resolution,
                "center_lat": centroid_lat,
                "center_lon": centroid_lon,
                "avg_edge_len": avg_edge_len,
                "cell_area": cell_area,
                "cell_perimeter": cell_perimeter,
                "cell_minimum": cell_minimum,
                "cell_maximum": cell_maximum,
                "cell_mean": cell_mean,
                "cell_median": cell_median,
                "cell_stddev": cell_stddev,
                "cell_pixel_count": cell_pixel_count,
                "geometry": cell_geom # if isinstance(cell_polygon, gpd.GeoDataFrame) and not cell_polygon.empty else None,
            }
            properties.append(cell_props)
        except Exception:
            continue

    src.close()

    # Build GeoDataFrame
    gdf = gpd.GeoDataFrame(properties, geometry="geometry", crs="EPSG:4326")

    # Use centralized output utility
    base_name = os.path.splitext(os.path.basename(raster_path))[0]
    output_name = f"{base_name}2dggrid" if output_format is not None else None
    return convert_to_output_format(gdf, output_format, output_name)

In [18]:
# patch command to avoid massive outputs by underlying tools
import mock
import sys
import types
import tqdm as tqdm_module

tqdm_patched = types.ModuleType("tqdm")
tqdm_patched.tqdm = tqdm_module.autonotebook.tqdm

## Convert RCM ARD Data to DGGS

#### NOTE

⚠️ Requires patch: https://github.com/opengeoshub/vgrid/pull/57 if running the `vgrid` "raster2..." native operations
on non-geometric CRS (i.e.: PROJCRS) rasters.


In [19]:
output_cell_stats_formats = ["parquet"]  # "geojson" can be used for visualization, but slow and large duplicates for big datasets
output_cell_data_format = ".tif"
output_base_dir = "./outputs/manitoba_rcm_ard"
dggs_types_res = [
    # ("H3", lambda: _src, _res, _out_fmt, **__: raster2h3(_src, _res, _out_fmt), [7, 8]),  # H3 L8 hexagon dimension ~= ISEA7H L9
    # ("A5", lambda: _src, _res, _out_fmt, **__: raster2a5(_src, _res, _out_fmt), [9]),
    # ("rHEALPix", lambda: _src, _res, _out_fmt, **__: raster2rhealpix(_src, _res, _out_fmt), [8, 9]),
    # NOTE: raster2dggrid
    #   wrapper dispatches to the underlying "dggrid" tool that provide additional DGGRS
    #   IGEO7 = ISEA7H with 7Z address representation (https://dggrid4py.readthedocs.io/en/latest/usage.html)
    # WARNING:
    #   use dggrid BIN_POINT_VALS directly or another custom method instead of 'raster2dggrid'
    #   raster2dggrid is very slow since it computes every lat/lon=>cellId for every (x,y) pixel sequentially
    #   it also reprojects each of those (lat,lon) coordinates one by one
    #   raster2dggrid also incorrectly picks any first cellId match and drops all other values within that same cell
    #   depending on the cell size/resolution, this first value might not be representative at all!
    # (
    #     "IGEO7",
    #     lambda *_, **__: raster2dggrid(dggrid_instance, "IGEO7", *_, dggrid_input_address_type="Z7_STRING", **__),
    #     [8, 9]
    # ),
    (
        "IGEO7",
        lambda *_, **__: raster2dggs_custom(
            dggrid_instance,
            "ISEA7H",
            *_,
            # WARNING: following requires new DGGRID v8
            #dggrid_output_address_type="Z7_STRING",
            output_cell_label_type="OUTPUT_ADDRESS_TYPE",
            output_address_type="HIERNDX",
            output_hier_ndx_system="Z7",
            output_hier_ndx_form="DIGIT_STRING",
            **__,
        ),
        [7, 8, 9, 10, 12], # 15]  # pre-computed res=15 (~10m) closest to ~20m rasters retrieved from STAC
        [False, False, False, False, True],
    ),
]

dggs_bar = tqdm(dggs_types_res, desc="Processing DGGS Types")
for dggrs_operations in dggs_bar:
    dggrs, dggrs_builder, dggrs_resolutions = dggrs_operations[:3]
    dggrs_output_cell_data = [False] * len(dggrs_resolutions) if len(dggrs_operations) < 4 else dggrs_operations[3]
    dggs_bar.set_postfix(dggrs=dggrs)
    res_bar = tqdm(
        zip(dggrs_resolutions, dggrs_output_cell_data),
        desc="Processing Resolutions",
        leave=False,
        total=len(dggrs_resolutions),
    )
    for resolution, output_cell_data in res_bar:
        res_bar.set_postfix(dggrs=dggrs, resolution=resolution)
        stac_items_bar = tqdm(result.items(), desc="Processing STAC Items", leave=False, total=result.matched())
        for item in stac_items_bar:
            stac_items_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=item.id)

            # backup source for later cross-reference
            date = item.properties["datetime"][:10]  # YYYY-MM-DD
            stac_item_dir = os.path.join(output_base_dir, "src", date)
            stac_item_path = os.path.join(stac_item_dir, f"stac_item_{item.id}.geojson")
            os.makedirs(stac_item_dir, exist_ok=True)
            with open(stac_item_path, mode="w", encoding="utf-8") as f:
                json.dump(item.to_dict(), f, indent=2)

            for band in ["rl", "rr"]:
                src_uri = item.assets[band].href
                src_name = os.path.basename(src_uri)
                res_bar.set_postfix(dggrs=dggrs, resolution=resolution, asset=src_name)

                src_dir = os.path.join(stac_item_dir, band)
                src_path = os.path.join(src_dir, src_name)
                out_dir = os.path.join(output_base_dir, dggrs, f"L{resolution}", date, band)
                os.makedirs(out_dir, exist_ok=True)
                out_files = {
                    fmt: os.path.join(out_dir, f"{os.path.splitext(src_name)[0]}.{fmt}")
                    for fmt in output_cell_stats_formats
                }
                if all(os.path.isfile(p) for p in out_files.values()):
                    print(f"  Skipping... All output files already exist: {list(out_files.values())}")
                    continue

                print(f"Converting to {dggrs}-L{resolution} [band={band}]. Saving to [{out_dir}]...")
                if not os.path.isfile(src_path):
                    os.makedirs(src_dir, exist_ok=True)
                    with mock.patch.dict(sys.modules, {"tqdm": tqdm_patched}):
                        download_file(src_uri, src_path)
                out_df = dggrs_builder(
                    src_path,
                    resolution=resolution,
                    output_format="gpd",  # GeoPandas Dataframe
                    output_cell_path=out_dir,  # only custom function
                    output_cell_data=output_cell_data,  # only custom function
                )
                # save results
                for fmt, out_path in out_files.items():
                    convert_to_output_format(out_df, output_format=fmt, output_name=out_path)

                # cleanup to save space
                if os.path.isfile(src_path):
                    os.remove(src_path)
                reproject_path = src_path.replace(".tif", "_EPSG4326.tif")
                if os.path.isfile(reproject_path):
                    os.remove(reproject_path)

Processing DGGS Types:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Resolutions:   0%|          | 0/5 [00:00<?, ?it/s]

Processing STAC Items:   0%|          | 0/89 [00:00<?, ?it/s]

  Skipping... All output files already exist: ['./outputs/manitoba_rcm_ard/IGEO7/L7/2025-09-16/rl/RCM3_OK3556292_PK3810344_1_SC30MCPD_20250916_123959_RL.parquet']
  Skipping... All output files already exist: ['./outputs/manitoba_rcm_ard/IGEO7/L7/2025-09-16/rr/RCM3_OK3556292_PK3810344_1_SC30MCPD_20250916_123959_RR.parquet']
  Skipping... All output files already exist: ['./outputs/manitoba_rcm_ard/IGEO7/L7/2025-09-13/rl/RCM2_OK3556292_PK3804068_2_SC30MCPB_20250913_124733_RL.parquet']
  Skipping... All output files already exist: ['./outputs/manitoba_rcm_ard/IGEO7/L7/2025-09-13/rr/RCM2_OK3556292_PK3804068_2_SC30MCPB_20250913_124733_RR.parquet']
  Skipping... All output files already exist: ['./outputs/manitoba_rcm_ard/IGEO7/L7/2025-09-13/rl/RCM2_OK3555385_PK3804066_1_SC30MCPB_20250913_124733_RL.parquet']
  Skipping... All output files already exist: ['./outputs/manitoba_rcm_ard/IGEO7/L7/2025-09-13/rr/RCM2_OK3555385_PK3804066_1_SC30MCPB_20250913_124733_RR.parquet']
  Skipping... All outp

Processing STAC Items:   0%|          | 0/89 [00:00<?, ?it/s]

...

Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/IGEO7/L10/2025-08-30/rl/RCM2_OK3556292_PK3775534_5_SC30MCPD_20250830_122959_RL.parquet
Converting to IGEO7-L10 [band=rr]. Saving to [./outputs/manitoba_rcm_ard/IGEO7/L10/2025-08-30/rr]...
** executing DGGRID version 8.42 with GDAL version 3110300 **
type sizes: big int: 64 bits / big double: 128 bits

** using meta file metafile_d377d80d-f85c-447d-967b-ad192ef265a7...
* parameter values:
dggrid_operation GENERATE_GRID (user set)
rng_type RAND (default)
precision 7 (default)
verbosity 0 (default)
pause_on_startup false (default)
pause_before_exit false (default)
update_frequency 100000 (default)
dggs_type ISEA7H (user set)
dggs_topology HEXAGON (user set)
dggs_proj ISEA (user set)
dggs_aperture_type PURE (user set)
dggs_aperture 7 (user set)
proj_datum WGS84_AUTHALIC_SPHERE (default)
dggs_orient_specify_type SPECIFIED (user set)
dggs_num_placements 1 (user set)
dggs_vert0_lon 11.2

/home/francis/dev/miniconda/envs/pydggsapi/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver FlatGeobuf does not support open option DRIVER
  return ogr_read(


Matched cells: 131763 (res=10) in POLYGON ((-94.85016594159757 48.51601142276877, -94.85016594159757 49.85717472190171, -97.03167386616124 49.85717472190171, -97.03167386616124 48.51601142276877, -94.85016594159757 48.51601142276877))


Converting raster to DGGRID:   0%|          | 0/131763 [00:00<?, ? cells/s]


KeyboardInterrupt



## Combine Results

Since we have generated separate dataset for each (DGGRS, resolution, date, band) combination,
we can aggregate those results to facilitate retrieval.